In [114]:
from xml.dom.minidom import (parse, Element)

In [115]:
corpus_filename = 'Tselina'
#corpus_filename = 'pedagogika'
#corpus_filename = 'sirija'

file_ext = '.xml'
doc = parse(corpus_filename + file_ext)
sentences = doc.getElementsByTagName('S') # все элементы с тегом S (все предложения корпуса)

In [116]:
f'Количество предложений в корпусе = {len(sentences)}'

'Количество предложений в корпусе = 79'

In [117]:
class Sentence:
    def __init__(self, sent: Element):
        self.id = sent.getAttribute('ID')
        self.words = []
        
        self.parseWords(sent.getElementsByTagName('W'))
        # получаем все элементы xml с тегом W (слова)
        
    def parseWords(self, rawWords):
        
        self.wordMap = {} # для последующего связывания слов
        # ключ - доминатор(айди доминатора) этого слова
        # значение - список слов, для которых доминатор равен ключу
        
        for rawWord in rawWords:
            
            word = Word(rawWord) # переносим в класс
            
            self.words.append(word) # добавляем в общий список слов
            
            if word.dom == '_root':
                self.rootWord = word # корневое слово
                
            # добавление в мапу
            if word.dom in self.wordMap.keys():
                self.wordMap[word.dom].append(word)
            else:
                self.wordMap[word.dom] = [word]

        # цикл по всем словам, чтобы связать их с помощью мапы
        for word in self.words:
            if word.id in self.wordMap.keys():
                for d in self.wordMap[word.id]:
                    word.addWord(d)
    def printGram(self):
        # печать грамматики предложения = печать грамматики одного слова(которое печатает все свои связанные слова, то есть все слова в предложении)
        self.rootWord.printGram() 
        
        
class Word:
    def __init__(self, w: Element):
        
        self.dom = w.getAttribute('DOM')
        self.feat = w.getAttribute('FEAT')
        self.id = w.getAttribute('ID')
        self.lemma = w.getAttribute('LEMMA')
        self.link = w.getAttribute('LINK')
        
        self.connectedWords = [] #связанные с этим словом слова
        
    def addWord(self, w):
        # добавление слова в связанные с этим словом слова
        self.connectedWords.append(w)
        
    def printGram(self):
        
        print(f'F({self.lemma}) -> ', end = '') # F(t) ->
        
        if self.connectedWords == []:
            print(f'[{self.feat}] ', end = '') # когда узел - терминальный
            
        else:
            t = False # для расположения граммем слова в нужном месте
            for w in self.connectedWords:
                # проход по всем словам в связанных словах
                if not t and int(w.id) > int(self.id): 
                    # когда этого не происходило ранее и когда айди текущего связанного слова превысило айди самого слова
                    print(f'[{self.feat}] ', end = '')
                    t = True
                    
                print(f'D({w.lemma}, {w.link}) ', end = '') # D(t, s)
                
            for w in self.connectedWords:
                print(f'\nD({w.lemma}, {w.link}) -> F({w.lemma})') #D(t, s) -> F(t)
                #print() # раскомментировать, если не хотим печатать D(t, s) -> F(t)
                w.printGram()
                
            
        

In [118]:
def parseSentences():
    for sent in sentences:
        #пробегаемся по всем предложениям корпуса
        parseSentence(sent)

def parseSentence(sent: Element):
    sentence = Sentence(sent)
    sentence.printGram()

In [119]:
import sys
filename = corpus_filename + '_' + 'gram.out'

In [120]:
orig_stdout = sys.stdout
f = open(filename, 'w', encoding = 'utf-16')
sys.stdout = f
# перенаправление stdout в файл, чтобы все печаталось в файл

parseSentences()

# перенаправление обратно
sys.stdout = orig_stdout
f.close()

In [121]:
filename_sorted = corpus_filename + '_' + 'gram_sorted.out'

In [122]:
# сортировка и удаление дубликатов в файле
def printSortedNoDuplicatesFile():
    lines_seen = set()
    with open(filename, 'r', encoding = 'utf-16') as r:
        with open(filename_sorted, 'w', encoding = 'utf-16') as f:
            for line in sorted(r):
                if line not in lines_seen:
                    print(line, end = '', file = f)
                    lines_seen.add(line)

In [123]:
printSortedNoDuplicatesFile()